In [7]:
# automated ref adding for lineage-note.txt, written by Xu Zou
import bs4
from bs4 import BeautifulSoup
from urllib import request

# the number of milestone we start to read
start_milestone=1
start_sub_milestone=1
end_milestone=795
end_sub_milestone=13
def read_milestone(num,di='cov-lineages/pango-designation'):
    # or sars-cov-2 proj if it starts to add milestones
    url="https://github.com/"+di+"/milestone/"+str(num)
    res=request.urlopen(url)
    rp=res.read()
    rp=rp.decode('utf-8')
    html_soup=BeautifulSoup(rp,"html")
    title=html_soup.title
    progress_percent=html_soup.findAll(class_="progress-percent")[0].text
    #updated version, ignore those with 0% progress.
    if progress_percent=="0%":
        return []
    
    title=title.text.replace(',','')
   
    title=title.split()[:-3]
    
    return title



In [8]:
# read milestones 
milestone_end=False
milestone=start_milestone
milestone_list=[]
milestone_list_sub=[]
milestone_sub=start_sub_milestone
import os 
listd=os.listdir()


if not("autoref" in listd):
    os.mkdir("autoref")
lista=os.listdir("autoref")
# reading milestones is very slow, so we save already read milestones to autoref/milestones.txt
if ("milestone.txt" in lista):
    w=open("autoref/milestone.txt",'r')
    milestones=w.readlines()
    for line in milestones:
        linsp=line.split()
        if len(linsp)>=2:
            
            issue=int(linsp[0])
            name=linsp[1]
            if (linsp[1]!='·' and linsp[1]!='Milestone'):
                milestone_list.append([linsp[0],linsp[1]])
            # don't read twice for old milestones
            if issue>=milestone:
                milestone=issue+1
    w.close()

if ("milestone_sub.txt" in lista):
    w=open("autoref/milestone_sub.txt",'r')
    milestones=w.readlines()
    for line in milestones:
        linsp=line.split()
        if len(linsp)>=2:
            issue=int(linsp[0])
            name=linsp[1]
            if (linsp[1]!='·' and linsp[1]!='Milestone'):
                milestone_list_sub.append([linsp[0],linsp[1]])
            # don't read twice for old milestones
            if issue>=milestone_sub:
                milestone_sub=issue+1          
    w.close()  
#rewrite the txt

w=open("autoref/milestone.txt",'w',newline='\n')
for item in milestone_list:
    print(item[0],item[1],file=w)


while (milestone<=end_milestone):
    try:
        title=read_milestone(milestone)
        for item in title:
            milestone_list.append([milestone,item])
            print(milestone,item,file=w)
    except Exception as e:
        print(milestone,e)
        
    milestone+=1
    if milestone%10==0:
        print("Reading Milestone",milestone)

w.close()

w=open("autoref/milestone_sub.txt",'w',newline='\n')
for item in milestone_list_sub:
    print(item[0],item[1],file=w)

milestone_end=False
while (milestone_sub<=end_sub_milestone):
    try:
        title=read_milestone(milestone_sub,di="sars-cov-2-variants/lineage-proposals")
        for item in title:
            milestone_list_sub.append([milestone_sub,item])
            print(milestone_sub,item,file=w)
    except Exception as e:
        print(milestone_sub,e)
        
    milestone_sub+=1
    if milestone_sub%10==0:
        print("Reading Milestone in sars-cov-2-variants ",milestone)

w.close()

    

    

In [9]:
def read_milestone_issues(name,di='cov-lineages/pango-designation'):
    # or sars-cov-2 proj if it starts to add milestones
    
    # first, deal with multiple names
    to_add=all_milestones[name]
    actual_milestone='"'+item_to_milestone[to_add]+'"'
    url="https://github.com/"+di+"/issues?q=milestone:"+actual_milestone
    res=request.urlopen(url)
    rp=res.read()
    rp=rp.decode('utf-8')
    html_soup=BeautifulSoup(rp,"html")
    html=html_soup.find_all(class_="Link--primary v-align-middle no-underline h4 js-navigation-open markdown-title" )
    try:
        id_str=html[0]['id']
        outid=id_str.split("_")[1]
        return int(outid)
    except Exception as e:
        print(name,e)
        return 0



In [ ]:
# read lineage_notes.txt

def milestone2issue(num):
    return 0
w=open("lineage_notes.txt",'r')
lineage_notes=w.readlines()
w.close()
milestone_tobecheck=[]
#create a name-milestone match for milestones
all_milestones={}
item_to_milestone={}
for item in milestone_list:
    to_add="#"+str(item[0])
    all_milestones[item[1]]=to_add
    if to_add in item_to_milestone:
        item_to_milestone[to_add]+='+'+item[1]
    else:
        item_to_milestone[to_add]=item[1]
        
for item in milestone_list_sub:
    to_add='sars-cov-2-variants/lineage-proposals#'+str(item[0])
    all_milestones[item[1]]=to_add
    if to_add in item_to_milestone:
        item_to_milestone[to_add]+='+'+item[1]
    else:
        item_to_milestone[to_add]=item[1]

changed_items=0
missing_milestones=[]
for num in range(1,len(lineage_notes)):
    line=lineage_notes[num]
    linsp=line.split()
    if (len(linsp)>=2):
        title=linsp[0]
        # for all lineages without an issue label but with a milestone
        if not("#" in line):
            if title in all_milestones:
                # this time we need to deal with the milestone 
                if 'sars' in all_milestones[title]:
                    issue_num=read_milestone_issues(title,di='sars-cov-2-variants/lineage-proposals')
                    if issue_num!=0:
                        missing_milestones.append([title,'sars-cov-2-variants/lineage-proposals#'+str(issue_num)])
                        print(missing_milestones[-1])
                else:
                    issue_num=read_milestone_issues(title)
                    if issue_num!=0:
                        missing_milestones.append([title,'#'+str(issue_num)])
                        print(missing_milestones[-1])
                    
#rewrite lineage_notes.txt
w=open("autoref/missing_refs.txt",'w')
for item in missing_milestones:
    print(item[0],item[1],file=w)
w.close()
        


In [1]:
#final rewrite of lineage_notes.txt
w=open("autoref/missing_refs.txt",'r')
data=w.readlines()
w.close()
missing_refs={}
for line in data:
    linsp=line.split()
    missing_refs[linsp[0]]=linsp[1].replace('\n','')

w=open("lineage_notes.txt",'r')
data=w.readlines()
w.close()
for i in range(len(data)):
    linsp=data[i].split()
    if linsp[0] in missing_refs:
        data[i]=data[i].replace('\n','')+" From "+missing_refs[linsp[0]]+'\n'

w=open("lineage_notes.txt",'w',newline='\n')
w.writelines(data)
w.close()
